## iii)-5. Implementation of Decision Tree 
This file is the implementation of the Decision Tree. In this section, the group generated their own decision tree, and compare its accuracy with the accuracy of the decision tree generated by Sklearn pakage.



#### iii)-5-A Decision Tree Generated by Sklearn Package 
This section shows the decision tree generated by Sklearn package. The second unit shows the accuracy of the complete tree with all the features, and the third one shows the accuracy of the complete tree with only relevant features.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.grid_search import GridSearchCV
from sklearn import tree
from sklearn.metrics import accuracy_score
from operator import itemgetter

/home/fei/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/fei/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [21]:
'''
        Complete Decision Tree With All Features
'''
#--------------------Import and pre-processe data--------------------
alleletronicsdata = pd.read_csv('voice.csv')
# Split traing and testing set 
x_train, x_test, y_train, y_test = train_test_split(alleletronicsdata.iloc[:,0:-1],alleletronicsdata.iloc[:,-1], test_size=0.2,random_state=1)


#--------------------Train.Fit Decision Tree.--------------------
decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)

#--------------------Test trained tree.--------------------
print('In-Sample Accuracy:{:.2f}%'.format(decisiontree.score(x_train, y_train)*100))
print('Out-of-Sample Accuracy:{:.2f}%\n'.format(decisiontree.score(x_test, y_test)*100))

#--------------------Confusion Table.--------------------
# predict the testing data
y_predict_train = decisiontree.predict(x_train)
y_predict_test = decisiontree.predict(x_test)

# in-sample
confusionmtrx_train = pd.DataFrame(confusion_matrix(y_pred=y_predict_train, y_true=y_train), 
                                   index=['true_Male', 'true_Female'],columns=['pred_Male', 'pred_Female'])
print("In-Sample Confusion Table:\n",confusionmtrx_train)

#out-of-sample
confusionmtrx_test = pd.DataFrame(confusion_matrix(y_pred=y_predict_test, y_true=y_test), index=['true_Male', 'true_Female'],
                  columns=['pred_Male', 'pred_Female'])
print("Out-of-Sample Confusion Table:\n",confusionmtrx_test)

#--------------------Print trained tree--------------------
#The complete tree is too big to be shown here. 
#Please refer to the pdf file(Decision_Tree_All_Features.pdf) in the folder.
dot_data = tree.export_graphviz(decisiontree, out_file=None,  class_names=['male', 'female'], impurity=True, filled=True, rounded=True, special_characters=True) 
graph = graphviz.Source(dot_data) 
graph.render("Decision_Tree_All_Features")


In-Sample Accuracy:100.00%
Out-of-Sample Accuracy:78.26%

In-Sample Confusion Table:
              pred_Male  pred_Female
true_Male          179            0
true_Female          0          189
Out-of-Sample Confusion Table:
              pred_Male  pred_Female
true_Male           38           13
true_Female          7           34


'Decision_Tree_All_Features.pdf'

In [22]:
'''
        Complete Decision Tree With Only Relevant Features 
'''
#--------------------Reduce features--------------------
columns_to_use = [ 'IQR',  'meanfun','label']
alleletronicsdata = alleletronicsdata.loc[:, columns_to_use].copy()
x_train, x_test, y_train, y_test = train_test_split(alleletronicsdata.iloc[:,0:-1],alleletronicsdata.iloc[:,-1], test_size=0.2,random_state=1)

#--------------------Train.Fit Decision Tree.--------------------
decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)

#--------------------Test trained tree.--------------------
print('In-Sample Accuracy:{:.2f}%'.format(decisiontree.score(x_train, y_train)*100))
print('Out-of-Sample Accuracy:{:.2f}%\n'.format(decisiontree.score(x_test, y_test)*100))

#--------------------Confusion Table.--------------------
# predict the testing data
y_predict_train = decisiontree.predict(x_train)
y_predict_test = decisiontree.predict(x_test)

# in-sample
confusionmtrx_train = pd.DataFrame(confusion_matrix(y_pred=y_predict_train, y_true=y_train), 
                                   index=['true_Male', 'true_Female'],columns=['pred_Male', 'pred_Female'])
print("In-Sample Confusion Table:\n",confusionmtrx_train)

#out-of-sample
confusionmtrx_test = pd.DataFrame(confusion_matrix(y_pred=y_predict_test, y_true=y_test), index=['true_Male', 'true_Female'],
                  columns=['pred_Male', 'pred_Female'])
print("Out-of-Sample Confusion Table:\n",confusionmtrx_test)
#--------------------Print trained tree.--------------------
#The complete tree is too big to be shown here. 
#Please refer to the pdf file(Decision_Tree_Two_Features.pdf) in the folder.
dot_data = tree.export_graphviz(decisiontree, out_file=None,  class_names=['male', 'female'], impurity=True, filled=True, rounded=True, special_characters=True) 
graph = graphviz.Source(dot_data) 

In-Sample Accuracy:100.00%
Out-of-Sample Accuracy:83.70%

In-Sample Confusion Table:
              pred_Male  pred_Female
true_Male          179            0
true_Female          0          189
Out-of-Sample Confusion Table:
              pred_Male  pred_Female
true_Male           39           12
true_Female          3           38


#### iii)-5-B Decision Tree Generated Without Using Sklearn Package 
This section shows the decision tree written by the group. The first 9 cells are importation of packages, and definition of functions and class. The 10th is the training and testing of complete tree. The last code unit shows the pruning process of the tree, and the accuracy of pruned tree

In [4]:
import os
import subprocess
import random as R
import numpy as np
import pandas as pd
from subprocess import call
import math 
from sklearn.model_selection import train_test_split
import os
import subprocess
import random as R
import numpy as np
import pandas as pd

In [5]:
#Import the .csv data file 

#--------------------Fetch Data From File--------------------
def GetData(File):  
    if os.path.exists(File):  #if file exists 
        df = pd.read_csv(File, index_col=None)# read file
        df['label'] = df['label'].map({'male': 0, 'female': 1})  #Chaneg gender label to binary code 
        # Drop unecessary columns, adjust sequence:
        df = df.dropna(axis=0, how='any')   #Drop instances with "NaN" 
    else:
        print("File not found.")
    return df

In [6]:
# Given a dataset with one independent attribute and a the dependent attribute, gender label, and a threshold, 
# this function returns the gini index of the sub sets splited based on the threshold. 
# Gini index is chosen over information entropy here, because log operation has a higher complexity. 

#------------------CalCulate GiniIndex------------------
def GiniIndex(DataSet,Th):
    gini=1.0       # initialize gini index to be 1
    m,n=DataSet.shape  

    #Counters       
    Count_L_M=0.0  #Counters for Male with attribute lower that threshold
    Count_L_F=0.0  #Counters for Female with attribute lower that threshold
    Count_H_M=0.0  #Counters for Male with attribute higher that threshold
    Count_H_F=0.0  #Counters for Female with attribute higher that threshold

    #A: The independent attribute; B: Gender label 
    A=DataSet['A']
    B=DataSet['B']
    
    # Count instances fall into each class(M/F) in each side (H?L) 
    for i in range(0,m):
        if A[i]<Th:
            if B[i]==0:
                Count_L_M+=1
            else:
                Count_L_F+=1
        else:
            if B[i]==0:
                Count_H_M+=1
            else:
                Count_H_F+=1
    
    TotalLow=Count_L_M+Count_L_F
    TotalHigh=Count_H_M+Count_H_F
    
    
    P_L_M=Count_L_M/TotalLow
    P_L_F=Count_L_F/TotalLow
    P_H_M=Count_H_M/TotalHigh
    P_H_F=Count_H_F/TotalHigh

    gini=P_L_M*(1-P_L_M)+P_L_F*(1-P_L_F)+P_H_M*(1-P_H_M)+P_H_F*(1-P_H_F)

    return gini

In [7]:
# Given a dataset with one independent attribute and a the dependent attribute, 
# this function returns the best splitting point of this attribute, and the gini 
# index of this split. GiniIndex() function is called in this function.
#------------------Find the Best Point to Split------------------
def FindSplitPoint(DataSet):   
    Th=0.0
    gini=1
    
    Min=DataSet['A'].min()
    Max=DataSet['A'].max()
    step=(Max-Min)/100.0  
    t=Min+step
    
    while t<Max:
        g=GiniIndex(DataSet,t)
        if g<gini:
            gini=g
            Th=t
        else:
            pass
        t+=step    
    return Th,gini

#---------Function Test-------
#print(FindSplitPoint(DataSet))


In [8]:
#Given the whole dataset, the best attribute to split this data set and the splitting threshold.

#------------------Split Data According to chosen Attribute------------------
def SplitData(DataSet,Attribute,Th):
    LowSide=DataSet[DataSet[Attribute]<Th]
    HighSide=DataSet[DataSet[Attribute]>=Th]
    return LowSide,HighSide

#--------Function Test-------------
#L,H=SplitData(df,'sd',0.096502470739196294)
#print(L)
#print(H)

In [9]:
##------------------Define Class node------------------
class node:             #Nodes in the tree
    def __init__(self,Data):
        self.name=None
        self.left=None
        self.right=None
        self.Threshold = None
        self.prev=None
        self.data=Data
        
    def prev(self):
        return self.prev
                   
    def AddLeft(self,NewData):    # add left child to node
        node1=node(NewData)
        self.left=node1
        node1.prev=self
        return node1
    
    def AddRight(self,NewData):   # add right child to node
        node1=node(NewData)
        self.right=node1
        node1.prev=self
        return node1

In [10]:
#Given the root node of the decision tree(in which the whole data set is included), this function will 
#grow the decision tree based on the data set. FindSplitPoint() is called in this function. 
#In addition, this is an iteration function.

#------------------ Grow Tree Given Root Node ------------------
def Grow(node):
    m,n=node.data.shape
       
    # The tree should stop growing if the labe is all in one gender or there is no independent attribute in data
    Purity=node.data['label'].sum()     #label: MAle=0;Female=1
    Flag=(n>1) and (Purity>0) and (Purity<m) #Flag=True: grow tree; else： stop 
    #---------Stop Growing----------
    if Flag==False:      #Generate Leaf node with no children
        if (Purity<m/2):   # All or most of the instances are labeled "Male"
            node.name='Male'
        else:              # All or most of the instances are labeled "Female"
            node.name='Female'
    
    #---------Grow Tree-------------
    else:  
        #Find Best Attribute to split data
        gini=2
        Th=0
        Attribute=''
        
        for i in range(0,n-1):
            DataSet=pd.DataFrame({'A':node.data[node.data.columns[i]],'B':node.data['label']})
            t,g=FindSplitPoint(DataSet)
            if g<gini:     # Record this attribute if it has better ginin than the previous ones 
                gini=g
                Th=t
                Attribute=node.data.columns[i]
            else:
                pass
        
        # Split data and DROP PROCESSED ATTRIBUTE
        LeftData,RightData=SplitData(node.data,Attribute,Th)
        #DROP PROCESSED ATTRIBUTE
        LeftData.drop([Attribute],axis=1,inplace=True)
        RightData.drop([Attribute],axis=1,inplace=True)
        #Adjust index after the split
        index_L=np.linspace(0, len(LeftData.index)-1,num=len(LeftData.index), dtype=int)
        index_R=np.linspace(0, len(RightData.index)-1,num=len(RightData.index), dtype=int)
        LeftData.index=index_L
        RightData.index=index_R
        
        # Fix current node
        node.name=Attribute
        node.Threshold=Th
        
        # Add children nodes
        LeftNode=node.AddLeft(LeftData)
        RightNode=node.AddRight(RightData)
      
        # Iteration: Grow children node  
        Grow(LeftNode)
        Grow(RightNode)


In [11]:
#Print put the stricture of grown tree. For test only

#------------------Print Tree------------------
def PrintTree(Root,level):
    TAB="      "*level
    print(TAB,Root.name,"\n")
    if Root.left!=None:
          PrintTree(Root.left,level=level+1)
    if Root.right!=None:
          PrintTree(Root.right,level=level+1)
    

In [12]:
# Given the root of the grown decision tree and the test set, this function calculate the accuracy of the prediction, 
# and print out a confusion table 

#--------------------After-Train Prediction--------------------
def Predict(root,test):
    p,q=test.shape
    Counter_MM=0 #Reconginze Male as Male
    Counter_MF=0 #Reconginze Male as Female
    Counter_FM=0 #Reconginze Female as Male
    Counter_FF=0 #Reconginze Female as Female
    
    for i in range(0,p):  # for each new insatance, go over the whole tree t9 opredict its class
        
        #Go over Tree. Start with root node, end with leaf node
        Current=root
        while(Current.left!=None):
            if test[Current.name][i]<Current.Threshold:
                Current=Current.left
            else:
                Current=Current.right
        
        gender=Current.name
        if gender=='Male':
            if test['label'][i]==0:
                Counter_MM+=1
            else:
                Counter_FM+=1
        elif gender=='Female':
            if test['label'][i]==1:
                Counter_FF+=1
            else:
                Counter_MF+=1
   
    AccuracyScore=(Counter_MM+Counter_FF)/p

    print(AccuracyScore)
    print("Recogninze Male as Male:",Counter_MM)
    print("Recogninze Male as Female:",Counter_MF)
    print("Recogninze Female as Male:",Counter_FM)
    print("Recogninze Female as Female:",Counter_FF)
    

In [26]:
#---------------------Main Function ---------------------
# Import data from file
df=GetData("voice.csv") 

#Split the whole set into Train set and Test set
Train,Test =train_test_split(df,test_size=460-368, train_size=368, random_state=1)
#Adjust the index of Test and Train after the split 
index_train=np.linspace(0, len(Train.index)-1,num=len(Train.index), dtype=int)
index_test=np.linspace(0, len(Test.index)-1,num=len(Test.index), dtype=int)
Train.index=index_train
Test.index=index_test

#Creat root node for the tree, and then grow it 
root_Full=node(Train)
Grow(root_Full)  

#Test the grown treen with Test data
print("With all 20 attributes ")
print("Out of sample accuracy:")
Predict(root_Full,Test)
print("\nIn sample accuracy:")
Predict(root_Full,Train)

#-----------------Drop Irrelevant Attribute--------------------
df.drop(['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'skew', 'kurt',
       'sp.ent', 'sfm', 'mode', 'centroid', 'minfun', 'maxfun',
       'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx'],
        axis=1,inplace=True)
#Split the whole set into Train set and Test set
Train,Test =train_test_split(df,test_size=460-368, train_size=368, random_state=1)
#Adjust the index of Test and Train after the split 
index_train=np.linspace(0, len(Train.index)-1,num=len(Train.index), dtype=int)
index_test=np.linspace(0, len(Test.index)-1,num=len(Test.index), dtype=int)
Train.index=index_train
Test.index=index_test

#Creat root node for the tree, and then grow it 
root_Two=node(Train)
Grow(root_Two)    

#Test the grown treen with Test data
print("\n\nWith relevant attributes ")
print("Out of sample accuracy:")
Predict(root_Two,Test)
print("\nIn sample accuracy:")
Predict(root_Two,Train)
# Data Set For Function Test    
#DataSet=pd.DataFrame({'A':df['sd'],'B':df['label']})

/home/fei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/fei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


With all 20 attributes 
Out of sample accuracy:
0.8043478260869565
Recogninze Male as Male: 37
Recogninze Male as Female: 4
Recogninze Female as Male: 14
Recogninze Female as Female: 37

In sample accuracy:
0.9266304347826086
Recogninze Male as Male: 180
Recogninze Male as Female: 9
Recogninze Female as Male: 18
Recogninze Female as Female: 161


With relevant attributes 
Out of sample accuracy:
0.8478260869565217
Recogninze Male as Male: 38
Recogninze Male as Female: 3
Recogninze Female as Male: 11
Recogninze Female as Female: 40

In sample accuracy:
0.8967391304347826
Recogninze Male as Male: 173
Recogninze Male as Female: 16
Recogninze Female as Male: 22
Recogninze Female as Female: 157


In [27]:
#------------------Chi Square Pruning-------------
'''
    The group choose the threshold of Q to be 0.15, after observing a the few Q values of most of the nodes.
    P_chance, i.e., the chance that  the attribute is irrelevant, is set to be 70%. Any attribute with P_chance lower
    than this value (a Q < 0.15), will be pruned. The pruning process uses post-order traversal. The function also 
    printd out the attributes that get pruned.
'''

def ChiSquarePrune(root):

    def recurse(node):
        if node.left.left != None:
            recurse(node.left)
        if node.right.left!=None:
            recurse(node.right)
        
        S,ftr=node.data.shape         # S:total number of instances; ftr:total number of attributes
        S=float(S)
        p=float(node.data['label'].sum())    # p: total number of female
        n=S-p                         # Total number of male 
    
        Data_Low,Data_High=SplitData(node.data,node.name,node.Threshold)
        S_L=float(len(Data_Low))
        S_H=float(len(Data_High))
        p_L=float(Data_Low['label'].sum())
        p_H=float(Data_High['label'].sum())
        n_L=S_L-p_L
        n_H=S_H-p_H
    
        p_L_hat=p/S*S_L
        n_L_hat=n/S*S_L
        p_H_hat=p/S*S_H
        n_H_hat=n/S*S_H
    
        Q=pow((p_L-p_L_hat),2)/p_L_hat+pow((n_L-n_L_hat),2)/n_L_hat+pow((p_H-p_H_hat),2)/p_H_hat+pow((n_H-n_H_hat),2)/n_H_hat
        if Q<0.5:
            print(node.name,"pruned")
            if p>S/2:
                node.name='Female'
            else:
                node.name='Male'
                
            node.left=None
            node.right=None
            node.Th=None
        
        return
    
    recurse(root)

    
#-------------Prune the Complete Tree-------------
root_pruned=root_Full
ChiSquarePrune(root_pruned) 
PrintTree(root_pruned,0) #print pruned tree

# Retest in-saple and out-of-sample accuracy
df=GetData("voice.csv") 
#Split the whole set into Train set and Test set
Train,Test =train_test_split(df,test_size=460-368, train_size=368, random_state=1)
#Adjust the index of Test and Train after the split 
index_train=np.linspace(0, len(Train.index)-1,num=len(Train.index), dtype=int)
index_test=np.linspace(0, len(Test.index)-1,num=len(Test.index), dtype=int)
Train.index=index_train
Test.index=index_test
#Print Result
print("Pruned Tree:")
print("Out of sample accuracy:")
Predict(root_pruned,Test)
print("\nIn sample accuracy:")
Predict(root_pruned,Train)

sfm pruned
modindx pruned
dfrange pruned
maxdom pruned
mindom pruned
maxfun pruned
minfun pruned
centroid pruned
mode pruned
sp.ent pruned
skew pruned
IQR pruned
Q75 pruned
Q25 pruned
sd pruned
meanfreq pruned
maxdom pruned
dfrange pruned
mindom pruned
meandom pruned
Q25 pruned
centroid pruned
mode pruned
sfm pruned
sp.ent pruned
kurt pruned
skew pruned
IQR pruned
median pruned
sd pruned
meanfreq pruned
 meanfun 

       meandom 

             Male 

             Female 

       maxfun 

             Male 

             Q75 

                   modindx 

                         Female 

                         Male 

                   Male 

Pruned Tree:
Out of sample accuracy:
0.8152173913043478
Recogninze Male as Male: 38
Recogninze Male as Female: 3
Recogninze Female as Male: 14
Recogninze Female as Female: 37

In sample accuracy:
0.9157608695652174
Recogninze Male as Male: 179
Recogninze Male as Female: 10
Recogninze Female as Male: 21
Recogninze Female as Female: 158


## iv)-5. Discussion of Dicesion Tree 
Here are tables taht compare results:
Accuracy of complete tree: 

 |Out-Sample-20 Features|         
 |----------------------|
 | Sklearn | No Sklearn |
 | 78.26%  |   80.43%   |
 
 | In-Sample-20 Features|         
 |----------------------|
 | Sklearn | No Sklearn |
 | 100.0%  |   92.66%   |
 
The in-sample accuracy is significantly higher than the out-of sample accuracy, which means there is overfitting in the model. The group tried to solve this problem by reducing irrelevant features, and pruning the complete tree.

 |Out-Sample-2 Features |         
 |----------------------|
 | Sklearn | No Sklearn |
 | 83.70%  |   84.78%   |
 
 | In-Sample-2 Features |         
 |----------------------|
 | Sklearn | No Sklearn |
 | 100.0%  |   89.67%   |
 
Deleting irrelevant features improved the out-of-sample accuracy, but the in sample accuracy decreased. This is reasonable, as overfitting mainly causes by the model's attemption of fitting the inaccurate samples in the training set. The decrease in in-sample accuracy acctualy means that these inaccurate samples are excluded during training. The group also tried to prune the tree. Chi square test pruning is chosen here, simply because the group does not have sufficint data. 
 
 Accuracy of pruned tree:
 
 | In-Sample | Out-Sample |
 |-----------|------------|
 |   81.52%  |   91.58%   |

Both methods sucessfuly reduced overfitting, and raised the out-of-sample accuracy. Deleting all irrelevant features are more suceesful in improving this performance, but its shot coming is also very obvious. The relevance of each feature is evaluated by human after visualized the data, but the visulaization might hide some non-linear, and more complex relationship between the attributes and the classifycation. Pruning, on the other hand, is less agressive in deleting irrelevant features. In a simple task like this, in which most of the relationship are linear, directly deleting irrelevant features works better. 



